In [ ]:
import os
import numpy as np
import struct

def cnt_bad_blocks_to_ascii(file_name):
    """This function reads event data from a binary CNT file

    Parameters:

    - **file_name (str)**: The name of the binary CNT file you want to read.
    
    Returns:

    - **onset_list**: A list of onset times (in seconds) of bad blocks.
    - **duration_list**: A list of durations (in seconds) of bad blocks.
  """
    
    #Helper function to convert cnt time to seconds
    def convert_cnt_to_secs(cnt_vals):
        if cnt_vals < 3665:
            return 0
        return 7.142857142857144e-06 * cnt_vals + -0.02617857142855089
    
    #Check if file ends with .cnt
    if not file_name.endswith('.cnt'):
        raise ValueError("File must be a binary EEG file (.eeg).")

    if not os.path.isfile(file_name):
        raise ValueError(f"File {file_name} not found.")



    SETUP_NCHANNELS_OFFSET = 370
    SETUP_RATE_OFFSET = 376
    SETUP_NSAMPLES_OFFSET = 864
    with open(file_name, 'rb') as f:
        


        # # #try:
        # # # Read general part of the ERP header and set variables
        # # f.read(20) # skip revision number
        # # f.read(376) # skip the first 362 bytes
        f.seek(SETUP_NCHANNELS_OFFSET)
        chan = struct.unpack('<H', f.read(2))[0]  # number of channels
        f.seek(SETUP_RATE_OFFSET)
        rate = struct.unpack('<H', f.read(2))[0]  # sample rate (Hz)
        print(chan)
        print(rate)
     
        f.seek(SETUP_NSAMPLES_OFFSET)
        pnts = struct.unpack('<H', f.read(2))[0]  # number of points per waveform
        print(pnts)
        n_bytes = 2
        event_table_pos = (900 +
                           75 * int(chan) +
                           n_bytes * int(chan) * int(rate))
        
        f.seek(886) # Event Table Position
        eventoffset = struct.unpack("<i", f.read(4))[0]
        print(eventoffset)
        nsamples = (eventoffset - (900 + 75 * chan)) // (2 * chan)
        print('nsamples:', nsamples)
        f.seek(eventoffset)
        eventtype = f.read(1)[0]
        print(eventtype)
        eventsize = struct.unpack("<l", f.read(4))[0]
        print(eventsize)
        nevent = eventsize // (8 if eventtype == 1 else 19)
        offset_sec_list = []
        accept_list = []
        for i in range(nevent):
            if eventtype == 1:
                f.seek(eventoffset + 9 + i * 8 + 4)
                stim_type = struct.unpack('<H', f.read(2))[0]
                key_board = struct.unpack('<B', f.read(1))[0]
                KeyPad_Accept = struct.unpack('<B', f.read(1))[0]
            else:
                #HBclhhfccc
                f.seek(eventoffset + 9 + i * 19)
                print(f'event: {i}')
                stim_type = struct.unpack('<H', f.read(2))[0]
                f.read(1)
                # key_board = struct.unpack('<B', f.read(1))[0]
                key_pad_accept = struct.unpack('<c', f.read(1))[0]      
                offset = struct.unpack('<l', f.read(4))[0]
                print(f'stim_type: {stim_type}')
                # print(f'key_board: {key_board}')
                print(f'key_pad_accept: {key_pad_accept}')
                if 'xc0' in str(key_pad_accept):
                    print('bad')
                    accept_list.append('bad')
                    offset_sec_list.append(convert_cnt_to_secs(offset))
                if 'xd0' in str(key_pad_accept):
                    print('good')
                    accept_list.append('good')
                    offset_sec_list.append(convert_cnt_to_secs(offset))

                print(f'offset: {offset}')
                print(f'offset in seconds: {convert_cnt_to_secs(offset)}')

        if len(accept_list)%2 != 0:
            #Raise error if uneven number of good and bad events
            raise ValueError('Uneven number of good and bad events')
        onset_list = []
        duration_list = []
        #Find find first occurence of 'bad' in accept_list and store its index
        first_bad_index = accept_list.index('bad')
        for i in range(first_bad_index, len(accept_list), 2):
            onset_list.append(offset_sec_list[i])
            duration_list.append(offset_sec_list[i+1] - offset_sec_list[i])
        description_list = ['bad']*len(onset_list)
        return onset_list, duration_list, description_list